In [1]:
"""To annotate a few datapoints on the scatterplot, inspect the PPV values and create txt files with the datapoints x and y values"""

'To annotate a few datapoints on the scatterplot, inspect the PPV values and create txt files with the datapoints x and y values'

In [2]:
import pandas as pd
import analysis_functions
import pickle
import urllib.parse
import urllib.request

In [3]:
pd.set_option('display.max_rows',1500)
pd.set_option('display.max_columns',1500)

In [4]:
basedir = '/scratch/ias41/ae_code'

In [6]:
#dest_dir = 'unbound_margin_pred_faers_vs_sider'
dest_dir = 'unbound_margin_pred_faers_vs_sider'

In [7]:
# Files and info needed for the analysis

# Target information
target_info = pd.read_csv(basedir + '/ae_target_links/data/target_names.txt', sep='\t')
target_info = target_info.loc[target_info['accession_organism']=='Homo sapiens',:]

# Target classification
chembl_target_classification = pd.read_csv(basedir + '/analysis/data/target_classification_all_levels_r.txt', sep = '\t')

# MedDRA hierchy
meddra_hier = pd.read_excel(basedir + '/analysis/data/all_faers_and_sider_aes_hier_output.xlsx', skiprows=4)
meddra_hier_selection = meddra_hier.loc[meddra_hier['Primary SOC']=='Y',[' Term','HLT','SOC','PT']].drop_duplicates()
meddra_hier_selection['HLT'] = meddra_hier_selection['HLT'].apply(lambda x: x.upper())

# Previously reported associations
# Known associations, merge with known hierarchy HLT
known_associations = pd.read_excel(basedir + '/prev_reported_safety_associations/data/safety_meddra_annotated_effects.xls')
known_associations['Annotated MedDRA PT'] = known_associations['Annotated MedDRA PT'].apply(lambda x: x.upper())
known_meddra_hier = pd.read_excel(basedir + '/prev_reported_safety_associations/data/safety_meddra_annotated_effects_for_hierarchy_output.xlsx', skiprows=4)
known_meddra_hier['PT'] = known_meddra_hier['PT'].apply(lambda x: x.upper())
known_meddra_hier[' Term'] = known_meddra_hier[' Term'].apply(lambda x: x.upper())
known_meddra_hier['HLT'] = known_meddra_hier['HLT'].apply(lambda x: x.upper())
known_meddra_hier_selection = known_meddra_hier.loc[known_meddra_hier['Primary SOC']=='Y',['PT','HLT',' Term']].drop_duplicates()
known_merged = known_associations.merge(known_meddra_hier_selection, left_on='Annotated MedDRA PT', right_on=' Term')

hlt_manual = pd.read_excel(basedir + '/prev_reported_safety_associations/data/safety_meddra_manually_annotated_hlt_effects.xls', index=False)
hlt_manual.rename(columns={'Annotated MedDRA HLT': 'HLT'}, inplace=True)
hlt_manual['HLT'] = hlt_manual['HLT'].apply(lambda x: x.upper())
hlt_manual.drop(columns=['Annotated MedDRA HLT Code'])

known_merged = pd.concat([known_merged, hlt_manual], sort=False).reset_index(drop=True)


In [11]:
known_hlt_tuples = set([(x[1]['Accession'], x[1]['HLT']) for x in known_merged.iterrows()])

In [12]:
known_pt_tuples = set([(x[1]['Accession'], x[1]['PT']) for x in known_merged.loc[~known_merged['PT'].isnull()].iterrows()])

In [13]:
with open(basedir + '/analysis/data/dirs_info.pkl', 'rb') as f:
    dirs = pickle.load(f)

In [14]:
faers_data = dirs['20200110_faers_unbound_margin_pred_005_PRR2']
sider_data = dirs['20200110_sider_unbound_margin_pred']
#faers_data = dirs['20200110_faers_cutoff6_pred_005_PRR2']
#sider_data = dirs['20200110_sider_cutoff6_pred']

# Load main info from directories
faers_pos, faers_sign =  analysis_functions.find_associations(basedir + '/ae_target_links/output/' + faers_data['dir'], min_n=faers_data['min_n'], lr=faers_data['lr'], pv=faers_data['pv'], target_info=target_info)
sider_pos, sider_sign = analysis_functions.find_associations(basedir + '/ae_target_links/output/' + sider_data['dir'], min_n=sider_data['min_n'], lr=sider_data['lr'], pv=sider_data['pv'], target_info=target_info)

# Calculate Positive predictive value
faers_sign['PPV'] = faers_sign.apply(lambda x: analysis_functions.calculate_ppv(x), axis=1)
sider_sign['PPV'] = sider_sign.apply(lambda x: analysis_functions.calculate_ppv(x), axis=1)

/scratch/ias41/envs/release/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/scratch/ias41/envs/release/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [15]:
def calculate_prevalence(x):
    ae_prevalence = (x['nr compounds with AE'] / x['nr compounds'])
    return ae_prevalence

In [16]:
sider_pos['ae_prevalence'] = sider_pos.apply(calculate_prevalence, axis=1)
sider_sign['ae_prevalence'] = sider_sign.apply(calculate_prevalence, axis=1)
faers_pos['ae_prevalence'] = faers_pos.apply(calculate_prevalence, axis=1)
faers_sign['ae_prevalence'] = faers_sign.apply(calculate_prevalence, axis=1)

/scratch/ias41/envs/release/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/scratch/ias41/envs/release/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [17]:
def calculate_improvement(x): 
    improvement =x['PPV'] - x['ae_prevalence']
    return improvement

In [19]:
for df in [faers_sign, sider_sign]:
    df['Value-added PPV'] = df.apply(calculate_improvement, axis=1)

/scratch/ias41/envs/release/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
# Calculate Positive predictive value
faers_merged = faers_sign.merge(meddra_hier_selection, left_on='Adverse Event', right_on=' Term')
sider_merged = sider_sign.merge(meddra_hier_selection, left_on='Adverse Event', right_on=' Term')

faers_merged['dataset'] = 'FAERS'
sider_merged['dataset'] = 'SIDER'

both_sign = pd.concat([faers_merged, sider_merged]).sort_values(by=['Value-added PPV', 'ae_hit_rate'], ascending=[False, False])[['Likelihood Ratio', 'nr compounds with AE','ae_hit_rate','PPV','Value-added PPV','Adverse Event','pref_name', 'dataset', 'accession', 'HLT','PT']].drop_duplicates().reset_index(drop=True)
both_sign['PT'] = both_sign['PT'].apply(lambda x: x.upper())

In [27]:
def find_hlt_known(row):
    if ((row['accession'],row['HLT'])) in known_hlt_tuples:
        return 1
    else:
        return 0
def find_pt_known(row):
    if ((row['accession'],row['PT'])) in known_pt_tuples:
        return 1
    else:
        return 0

In [28]:
both_sign['hlt_known'] = both_sign.apply(find_hlt_known, axis=1)

In [29]:
both_sign['pt_known'] = both_sign.apply(find_pt_known, axis=1)

In [30]:
both_sign.head()

,Likelihood Ratio,nr compounds with AE,ae_hit_rate,PPV,Value-added PPV,Adverse Event,pref_name,dataset,accession,HLT,PT,hlt_known,pt_known
0,71.333333,27,0.333333,0.818182,0.758841,DUODENAL ULCER,Cyclooxygenase-1,FAERS,P23219,DUODENAL ULCERS AND PERFORATION,DUODENAL ULCER,0,0
1,58.666667,15,0.333333,0.833333,0.754799,GLYCOSURIA,Carbonic anhydrase VB,SIDER,Q9Y2D0,URINARY ABNORMALITIES,GLYCOSURIA,0,0
2,42.750000,20,0.250000,0.833333,0.728621,JAUNDICE CHOLESTATIC,Carbonic anhydrase VB,SIDER,Q9Y2D0,CHOLESTASIS AND JAUNDICE,JAUNDICE CHOLESTATIC,0,0
3,40.476190,21,0.238095,0.833333,0.723386,APLASTIC ANAEMIA,Carbonic anhydrase VB,SIDER,Q9Y2D0,MARROW DEPRESSION AND HYPOPLASTIC ANAEMIAS,APLASTIC ANAEMIA,0,0
4,46.666667,24,0.166667,0.800000,0.721053,HYPERKALAEMIA,Type-1 angiotensin II receptor,SIDER,P30556,POTASSIUM IMBALANCE,HYPERKALAEMIA,0,0


In [15]:
both_sign

,Likelihood Ratio,nr compounds with AE,ae_hit_rate,PPV,Adverse Event,pref_name,dataset,accession,HLT,PT,hlt_known,pt_known
0,inf,7,0.714286,1.000000,DYSPEPSIA,Tyrosine-protein kinase JAK2,FAERS,O60674,DYSPEPTIC SIGNS AND SYMPTOMS,DYSPEPSIA,0,0
1,inf,12,0.416667,1.000000,DYSPEPSIA,Tyrosine-protein kinase TYK2,FAERS,P29597,DYSPEPTIC SIGNS AND SYMPTOMS,DYSPEPSIA,0,0
2,inf,13,0.384615,1.000000,HYPOTHYROIDISM,Vascular endothelial growth factor receptor 1,SIDER,P17948,THYROID HYPOFUNCTION DISORDERS,HYPOTHYROIDISM,0,0
3,inf,42,0.166667,1.000000,HYPONATRAEMIA,Carbonic anhydrase IV,SIDER,P22748,SODIUM IMBALANCE,HYPONATRAEMIA,0,0
4,inf,37,0.135135,1.000000,DRUG EXPOSURE DURING PREGNANCY,Cytochrome P450 11B1,FAERS,P15538,"EXPOSURES ASSOCIATED WITH PREGNANCY, DELIVERY ...",EXPOSURE DURING PREGNANCY,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3242,12.561702,5,0.800000,0.078431,DEATH OF RELATIVE,Serotonin transporter,FAERS,P31645,BEREAVEMENT ISSUES,DEATH OF RELATIVE,0,0
3243,12.561702,5,0.800000,0.078431,FEELING GUILTY,Serotonin transporter,FAERS,P31645,MOOD ALTERATIONS WITH DEPRESSIVE SYMPTOMS,FEELING GUILTY,0,0
3244,10.453901,6,0.666667,0.078431,EXTRADURAL HAEMATOMA,Serotonin transporter,FAERS,P31645,CEREBRAL INJURIES NEC,EXTRADURAL HAEMATOMA,0,0
3245,8.559719,7,0.714286,0.075758,HYPERPROLACTINAEMIA,Serotonin 2a (5-HT2a) receptor,FAERS,P28223,ANTERIOR PITUITARY HYPERFUNCTION,HYPERPROLACTINAEMIA,0,0


In [16]:
both_sign.loc[(both_sign['hlt_known']==1)]

,Likelihood Ratio,nr compounds with AE,ae_hit_rate,PPV,Adverse Event,pref_name,dataset,accession,HLT,PT,hlt_known,pt_known
147,2.945223,314,0.101911,0.761905,DIZZINESS,Serotonin 2a (5-HT2a) receptor,SIDER,P28223,NEUROLOGICAL SIGNS AND SYMPTOMS NEC,DIZZINESS,1,0
193,18.136364,44,0.159091,0.700000,ATAXIA,GABA receptor alpha-1 subunit,SIDER,P14867,COORDINATION AND BALANCE DISTURBANCES,ATAXIA,1,1
263,5.322657,193,0.129534,0.666667,SOMNOLENCE,Histamine H1 receptor,SIDER,P35367,DISTURBANCES IN CONSCIOUSNESS NEC,SOMNOLENCE,1,0
318,5.413793,174,0.114943,0.633333,TACHYCARDIA,Muscarinic acetylcholine receptor M2,SIDER,P08172,RATE AND RHYTHM DISORDERS NEC,TACHYCARDIA,1,1
345,4.152299,116,0.172414,0.625000,TACHYCARDIA,Muscarinic acetylcholine receptor M1,SIDER,P11229,RATE AND RHYTHM DISORDERS NEC,TACHYCARDIA,1,1
378,4.187500,176,0.125000,0.611111,TACHYCARDIA,Histamine H1 receptor,SIDER,P35367,RATE AND RHYTHM DISORDERS NEC,TACHYCARDIA,1,0
541,24.818182,22,0.272727,0.600000,DEPRESSED LEVEL OF CONSCIOUSNESS,GABA receptor alpha-1 subunit,SIDER,P14867,DISTURBANCES IN CONSCIOUSNESS NEC,DEPRESSED LEVEL OF CONSCIOUSNESS,1,0
643,15.937500,56,0.107143,0.600000,GYNAECOMASTIA,Androgen Receptor,SIDER,P10275,BREAST DISORDERS NEC,GYNAECOMASTIA,1,1
660,21.042857,35,0.085714,0.600000,RENAL FAILURE ACUTE,Angiotensin II type 2 (AT-2) receptor,SIDER,P50052,RENAL FAILURE AND IMPAIRMENT,ACUTE KIDNEY INJURY,1,0
797,4.940171,135,0.125926,0.566667,VISION BLURRED,Muscarinic acetylcholine receptor M2,SIDER,P08172,VISUAL DISORDERS NEC,VISION BLURRED,1,1


In [194]:
both_sign.loc[both_sign['Adverse Event'].str.contains('OCULO')]

,Likelihood Ratio,nr compounds with AE,ae_hit_rate,PPV,Adverse Event,pref_name,dataset,accession,HLT,PT,hlt_known,pt_known,interest,novel_interest
323,19.714286,7,0.428571,0.272727,OCULOGYRIC CRISIS,Alpha-1a adrenergic receptor,FAERS,P35348,OCULAR NERVE AND MUSCLE DISORDERS,OCULOGYRIC CRISIS,0,0,0,0
324,33.910714,7,0.428571,0.272727,OCULOGYRIC CRISIS,Dopamine D4 receptor,FAERS,P21917,OCULAR NERVE AND MUSCLE DISORDERS,OCULOGYRIC CRISIS,0,0,0,0
337,16.636364,8,0.500000,0.266667,OCULOGYRIC CRISIS,Dopamine D2 receptor,SIDER,P14416,OCULAR NERVE AND MUSCLE DISORDERS,OCULOGYRIC CRISIS,0,0,0,0
378,20.625000,8,0.250000,0.250000,OCULOGYRIC CRISIS,Muscarinic acetylcholine receptor M3,SIDER,P20309,OCULAR NERVE AND MUSCLE DISORDERS,OCULOGYRIC CRISIS,0,0,0,0
401,15.950000,6,0.500000,0.230769,OCULOGYRIC CRISIS,Serotonin 7 (5-HT7) receptor,FAERS,P34969,OCULAR NERVE AND MUSCLE DISORDERS,OCULOGYRIC CRISIS,0,0,0,0
406,15.342857,7,0.428571,0.230769,OCULOGYRIC CRISIS,Dopamine D3 receptor,SIDER,P35462,OCULAR NERVE AND MUSCLE DISORDERS,OCULOGYRIC CRISIS,0,0,0,0
513,11.577640,7,0.571429,0.148148,OCULOGYRIC CRISIS,Dopamine D2 receptor,FAERS,P14416,OCULAR NERVE AND MUSCLE DISORDERS,OCULOGYRIC CRISIS,0,0,0,0


In [150]:
both_sign.loc[both_sign['dataset']=='FAERS'].sort_values(by='PPV', ascending=False)

,Likelihood Ratio,nr compounds with AE,ae_hit_rate,PPV,Adverse Event,pref_name,dataset,accession,HLT,PT,hlt_known,pt_known
22,71.333333,27,0.333333,0.818182,DUODENAL ULCER,Cyclooxygenase-1,FAERS,P23219,DUODENAL ULCERS AND PERFORATION,DUODENAL ULCER,0,0
44,38.322581,31,0.258065,0.666667,DUODENAL ULCER,Cyclooxygenase-2,FAERS,P35354,DUODENAL ULCERS AND PERFORATION,DUODENAL ULCER,0,0
68,74.000000,6,0.500000,0.600000,LIP OEDEMA,Angiotensin-converting enzyme,FAERS,P12821,ORAL SOFT TISSUE SWELLING AND OEDEMA,LIP OEDEMA,0,0
69,48.833333,9,0.333333,0.600000,PANCREATITIS NECROTISING,Angiotensin-converting enzyme,FAERS,P12821,ACUTE AND CHRONIC PANCREATITIS,PANCREATITIS NECROTISING,0,0
70,43.800000,10,0.300000,0.600000,HYPOVOLAEMIC SHOCK,Angiotensin-converting enzyme,FAERS,P12821,CIRCULATORY COLLAPSE AND SHOCK,HYPOVOLAEMIC SHOCK,0,0
81,22.484211,19,0.315789,0.545455,DIABETIC NEPHROPATHY,Alpha-1a adrenergic receptor,FAERS,P35348,NEPHROPATHIES AND TUBULAR DISORDERS NEC,DIABETIC NEPHROPATHY,0,0
115,9.515625,40,0.175000,0.466667,TORSADE DE POINTES,HERG,FAERS,Q12809,VENTRICULAR ARRHYTHMIAS AND CARDIAC ARREST,TORSADE DE POINTES,0,0
117,13.804511,19,0.315789,0.461538,DIABETIC NEPHROPATHY,Serotonin 7 (5-HT7) receptor,FAERS,P34969,NEPHROPATHIES AND TUBULAR DISORDERS NEC,DIABETIC NEPHROPATHY,0,0
125,20.438596,19,0.263158,0.454545,RESPIRATORY DEPRESSION,Delta opioid receptor,FAERS,P41143,BREATHING ABNORMALITIES,RESPIRATORY DEPRESSION,1,1
128,12.241379,29,0.172414,0.454545,TUBULOINTERSTITIAL NEPHRITIS,Cyclooxygenase-1,FAERS,P23219,NEPHRITIS NEC,TUBULOINTERSTITIAL NEPHRITIS,0,0


In [31]:
def get_gene_name(accession):
    url = 'https://www.uniprot.org/uploadlists/'

    params = {
    'from': 'UNIPROTKB AC/ID',
    'to': 'GENENAME',
    'format': 'tab',
    'query': f'{accession}'
    }

    data = urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(url, data)
    with urllib.request.urlopen(req) as f:
        response = f.read().decode('utf-8')
    gene_id = response.split('\t')[2].strip('\n')
    return gene_id
    

In [32]:
gene_name_dict = dict()

In [41]:
def get_annotations(df, indeces_list):
    annotations = []
    for row in df.loc[indeces_list].iterrows():
        
        gene_name = get_gene_name(row[1]['accession'])
        gene_name_dict[row[1]['accession']] = (row[1]['pref_name'], gene_name)
        annotations.append([gene_name+'-\n'+row[1]['Adverse Event'].lower().capitalize(), row[1]['ae_hit_rate'], row[1]['Value-added PPV'], row[1]['ae_hit_rate'], row[1]['Value-added PPV']])
    return annotations

In [20]:
both_sign.loc[(both_sign['hlt_known']==1)&(both_sign['PPV']<0.3)]

,Likelihood Ratio,nr compounds with AE,ae_hit_rate,PPV,Adverse Event,pref_name,dataset,accession,HLT,PT,hlt_known,pt_known
2491,7.954887,28,0.285714,0.296296,MYDRIASIS,Muscarinic acetylcholine receptor M3,SIDER,P20309,PUPIL DISORDERS,MYDRIASIS,1,1
2493,7.234848,22,0.227273,0.294118,EXTRAPYRAMIDAL DISORDER,Dopamine D1 receptor,SIDER,P21728,DYSKINESIAS AND MOVEMENT DISORDERS NEC,EXTRAPYRAMIDAL DISORDER,1,0
2495,5.410319,37,0.162162,0.294118,ORTHOSTATIC HYPOTENSION,Dopamine D1 receptor,SIDER,P21728,VASCULAR HYPOTENSIVE DISORDERS,ORTHOSTATIC HYPOTENSION,1,0
2517,9.721739,23,0.260870,0.285714,MYDRIASIS,Norepinephrine transporter,SIDER,P23975,PUPIL DISORDERS,MYDRIASIS,1,0
2538,6.812253,22,0.409091,0.281250,MYDRIASIS,Muscarinic acetylcholine receptor M1,SIDER,P11229,PUPIL DISORDERS,MYDRIASIS,1,0
2539,4.039953,37,0.243243,0.281250,ORTHOSTATIC HYPOTENSION,Dopamine D2 receptor,SIDER,P14416,VASCULAR HYPOTENSIVE DISORDERS,ORTHOSTATIC HYPOTENSION,1,1
2577,2.888889,72,0.111111,0.266667,HYPERGLYCAEMIA,Glucocorticoid receptor,SIDER,P04150,HYPERGLYCAEMIC CONDITIONS NEC,HYPERGLYCAEMIA,1,1
2596,4.383117,22,0.227273,0.263158,BLOOD PRESSURE INCREASED,Angiotensin-converting enzyme,SIDER,P12821,VASCULAR TESTS NEC (INCL BLOOD PRESSURE),BLOOD PRESSURE INCREASED,1,0
2684,9.605263,19,0.526316,0.238095,NEUROLEPTIC MALIGNANT SYNDROME,Serotonin 2a (5-HT2a) receptor,SIDER,P28223,MUSCLE TONE ABNORMAL,NEUROLEPTIC MALIGNANT SYNDROME,1,0
2694,5.530303,33,0.303030,0.238095,DYSKINESIA,Serotonin 2c (5-HT2c) receptor,SIDER,P28335,DYSKINESIAS AND MOVEMENT DISORDERS NEC,DYSKINESIA,1,1


In [20]:
known_merged.loc[known_merged['PT']=='PARKINSONISM']

,Accession,ChEMBL target name,Original effect,Formatted effect,PubMed ID,Reference,Annotated MedDRA PT,Annotated MedDRA PT Code,PT,HLT,Term,Annotated MedDRA HLT Code
944,P21728,Dopamine D1 receptor,parkinsonian symptoms (tremors),Parkinsonian symptoms (tremors),PMID23197038,"Bowes J, Brown AJ, Hamon J, Jarolimek W, Sridh...",PARKINSONISM,10034010.0,PARKINSONISM,PARKINSON'S DISEASE AND PARKINSONISM,PARKINSONISM,NaN
945,P21728,Dopamine D1 receptor,parkinsonism,Parkinsonism,PMID28216264,"Lynch JJ 3rd, Van Vleet TR, Mittelstadt SW, Bl...",PARKINSONISM,10034010.0,PARKINSONISM,PARKINSON'S DISEASE AND PARKINSONISM,PARKINSONISM,NaN
946,Q01959,Dopamine transporter,parkinsonism,Parkinsonism,PMID23197038,"Bowes J, Brown AJ, Hamon J, Jarolimek W, Sridh...",PARKINSONISM,10034010.0,PARKINSONISM,PARKINSON'S DISEASE AND PARKINSONISM,PARKINSONISM,NaN
947,Q01959,Dopamine transporter,parkinsonism,Parkinsonism,PMID28216264,"Lynch JJ 3rd, Van Vleet TR, Mittelstadt SW, Bl...",PARKINSONISM,10034010.0,PARKINSONISM,PARKINSON'S DISEASE AND PARKINSONISM,PARKINSONISM,NaN


In [21]:
known_both = both_sign.loc[both_sign['pt_known']==1,:]

In [42]:
combs_of_interest = {('P14416', 'HYPERPROLACTINAEMIA'), ('Q12809', 'TORSADE DE POINTES'), ('P12821', 'PANCREATITIS NECROTISING'), ('P23219', 'DUODENAL ULCER'), ('P08172', 'TREMOR'), ('P20309','DRY MOUTH'), ('P28335','DYSKINESIA'), ('P35368', 'ORTHOSTATIC HYPOTENSION'), ('P21728','PARKINSONISM'), ('P41143','RESPIRATORY DEPRESSION'), ('P23975', 'URINARY RETENTION'), ('P28223', 'NEUROLEPTIC MALIGNANT SYNDROME'), ('P11229', 'MIOSIS')}

In [43]:
def is_of_interest(x, combs_list):
    if (x['accession'],x['Adverse Event']) in combs_list:
        return 1
    else:
        return 0
both_sign['interest'] = both_sign.apply(is_of_interest,combs_list=combs_of_interest, axis=1)

In [44]:
index_list = list(both_sign.loc[both_sign['interest']==1].index)

In [45]:
index_list

[0, 13, 34, 88, 102, 111, 114, 131, 144, 232, 308, 346, 490, 509]

In [46]:
annotations_known = get_annotations(both_sign,index_list)
annotations_known

[['PTGS1-\nDuodenal ulcer',
  0.3333333333333333,
  0.7588411588411589,
  0.3333333333333333,
  0.7588411588411589],
 ['CHRM2-\nTremor',
  0.05263157894736842,
  0.656084656084656,
  0.05263157894736842,
  0.656084656084656],
 ['ACE-\nPancreatitis necrotising',
  0.3333333333333333,
  0.5701986754966887,
  0.3333333333333333,
  0.5701986754966887],
 ['CHRM3-\nDry mouth',
  0.05555555555555555,
  0.4460735586481113,
  0.05555555555555555,
  0.4460735586481113],
 ['OPRD1-\nRespiratory depression',
  0.2631578947368421,
  0.41537019681349574,
  0.2631578947368421,
  0.41537019681349574],
 ['SLC6A2-\nUrinary retention',
  0.125,
  0.40551181102362205,
  0.125,
  0.40551181102362205],
 ['ADRA1B-\nOrthostatic hypotension',
  0.2380952380952381,
  0.40256525652565256,
  0.2380952380952381,
  0.40256525652565256],
 ['KCNH2-\nTorsade de pointes',
  0.175,
  0.3824561403508772,
  0.175,
  0.3824561403508772],
 ['DRD1-\nParkinsonism', 0.2, 0.3732620320855615, 0.2, 0.3732620320855615],
 ['HTR2C-\n

In [47]:
annotations_known_df = pd.DataFrame(annotations_known)
annotations_known_df.columns = ['Annotation', 'x', 'y', 'x_text', 'y_text']
annotations_known_df.to_csv(basedir + '/analysis/results/{}/annotations_known_va-PPV.txt'.format(dest_dir), sep='\t', index=False)

### Now for novel significant associations - instead of known associations

In [48]:
novel_sign = both_sign.loc[both_sign['hlt_known']==0]
novel_sign.head()

,Likelihood Ratio,nr compounds with AE,ae_hit_rate,PPV,Value-added PPV,Adverse Event,pref_name,dataset,accession,HLT,PT,hlt_known,pt_known,interest
0,71.333333,27,0.333333,0.818182,0.758841,DUODENAL ULCER,Cyclooxygenase-1,FAERS,P23219,DUODENAL ULCERS AND PERFORATION,DUODENAL ULCER,0,0,1
1,58.666667,15,0.333333,0.833333,0.754799,GLYCOSURIA,Carbonic anhydrase VB,SIDER,Q9Y2D0,URINARY ABNORMALITIES,GLYCOSURIA,0,0,0
2,42.750000,20,0.250000,0.833333,0.728621,JAUNDICE CHOLESTATIC,Carbonic anhydrase VB,SIDER,Q9Y2D0,CHOLESTASIS AND JAUNDICE,JAUNDICE CHOLESTATIC,0,0,0
3,40.476190,21,0.238095,0.833333,0.723386,APLASTIC ANAEMIA,Carbonic anhydrase VB,SIDER,Q9Y2D0,MARROW DEPRESSION AND HYPOPLASTIC ANAEMIAS,APLASTIC ANAEMIA,0,0,0
4,46.666667,24,0.166667,0.800000,0.721053,HYPERKALAEMIA,Type-1 angiotensin II receptor,SIDER,P30556,POTASSIUM IMBALANCE,HYPERKALAEMIA,0,0,0


In [30]:
both_sign.sort_values(by=['PPV','ae_hit_rate'], ascending=[False,False])

,Likelihood Ratio,nr compounds with AE,ae_hit_rate,PPV,Adverse Event,pref_name,dataset,accession,HLT,PT,hlt_known,pt_known,interest
0,inf,7,0.714286,1.000000,DYSPEPSIA,Tyrosine-protein kinase JAK2,FAERS,O60674,DYSPEPTIC SIGNS AND SYMPTOMS,DYSPEPSIA,0,0,0
1,inf,12,0.416667,1.000000,DYSPEPSIA,Tyrosine-protein kinase TYK2,FAERS,P29597,DYSPEPTIC SIGNS AND SYMPTOMS,DYSPEPSIA,0,0,0
2,inf,13,0.384615,1.000000,HYPOTHYROIDISM,Vascular endothelial growth factor receptor 1,SIDER,P17948,THYROID HYPOFUNCTION DISORDERS,HYPOTHYROIDISM,0,0,0
3,inf,42,0.166667,1.000000,HYPONATRAEMIA,Carbonic anhydrase IV,SIDER,P22748,SODIUM IMBALANCE,HYPONATRAEMIA,0,0,0
4,inf,37,0.135135,1.000000,DRUG EXPOSURE DURING PREGNANCY,Cytochrome P450 11B1,FAERS,P15538,"EXPOSURES ASSOCIATED WITH PREGNANCY, DELIVERY ...",EXPOSURE DURING PREGNANCY,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3242,12.561702,5,0.800000,0.078431,DEATH OF RELATIVE,Serotonin transporter,FAERS,P31645,BEREAVEMENT ISSUES,DEATH OF RELATIVE,0,0,0
3243,12.561702,5,0.800000,0.078431,FEELING GUILTY,Serotonin transporter,FAERS,P31645,MOOD ALTERATIONS WITH DEPRESSIVE SYMPTOMS,FEELING GUILTY,0,0,0
3244,10.453901,6,0.666667,0.078431,EXTRADURAL HAEMATOMA,Serotonin transporter,FAERS,P31645,CEREBRAL INJURIES NEC,EXTRADURAL HAEMATOMA,0,0,0
3245,8.559719,7,0.714286,0.075758,HYPERPROLACTINAEMIA,Serotonin 2a (5-HT2a) receptor,FAERS,P28223,ANTERIOR PITUITARY HYPERFUNCTION,HYPERPROLACTINAEMIA,0,0,0


In [49]:
novel_combs_of_interest = {('P35462', 'NEUROLEPTIC MALIGNANT SYNDROME'), ('P35368', 'DIABETIC NEPHROPATHY'), ('Q9Y2D0', 'LEUKOPENIA'), ('P08172', 'SOMNOLENCE'), ('P50406', 'CONSTIPATION'), ('Q9Y2D0', 'JAUNDICE CHOLESTATIC'), ('P31645', 'MYOCLONUS')} 

In [50]:
both_sign['novel_interest'] = both_sign.apply(is_of_interest, combs_list=novel_combs_of_interest, axis=1)

In [51]:
list(both_sign.loc[both_sign['novel_interest']==1].index)

[2, 7, 10, 27, 108, 171, 287, 421]

In [52]:
novel_indeces = list(both_sign.loc[both_sign['novel_interest']==1].index)

In [53]:
annotations_sign = get_annotations(both_sign, novel_indeces)
annotations_sign

[['CA5B-\nJaundice cholestatic',
  0.25,
  0.7286212914485166,
  0.25,
  0.7286212914485166],
 ['CA5B-\nLeukopenia',
  0.1111111111111111,
  0.7172774869109948,
  0.1111111111111111,
  0.7172774869109948],
 ['CHRM2-\nSomnolence',
  0.03888888888888889,
  0.6825396825396826,
  0.03888888888888889,
  0.6825396825396826],
 ['HTR6-\nConstipation',
  0.042857142857142864,
  0.6007326007326007,
  0.042857142857142864,
  0.6007326007326007],
 ['ADRA1B-\nDiabetic nephropathy',
  0.2777777777777778,
  0.40999099909991,
  0.2777777777777778,
  0.40999099909991],
 ['DRD3-\nNeuroleptic malignant syndrome',
  0.35714285714285715,
  0.346259220231823,
  0.35714285714285715,
  0.346259220231823],
 ['DRD3-\nNeuroleptic malignant syndrome',
  0.36,
  0.2676651305683564,
  0.36,
  0.2676651305683564],
 ['SLC6A4-\nMyoclonus',
  0.20512820512820512,
  0.19906300326972817,
  0.20512820512820512,
  0.19906300326972817]]

In [54]:
annotations_sign_df = pd.DataFrame(annotations_sign)
annotations_sign_df.columns = ['Annotation', 'x', 'y', 'x_text', 'y_text']
annotations_sign_df.to_csv(basedir + '/analysis/results/{}/annotations_sign_va-PPV.txt'.format(dest_dir), sep='\t', index=False)
annotations_sign_df.to_csv(basedir + '/analysis/results/{}/annotations_sign_custom_va-PPV.txt'.format(dest_dir), sep='\t', index=False)

In [37]:
with open(basedir + '/analysis/results/{}/gene_names.txt'.format(dest_dir), 'w') as f:
    f.write(', '.join([gene_name_dict[item][1]+': '+gene_name_dict[item][0] for item in gene_name_dict]))